In [1]:
import os

from dotenv import load_dotenv
from etg import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

# Search Parameters
CITY = "Москва"

CHECKIN_DATE = "2026-02-02"
CHECKOUT_DATE = "2026-02-04"

CURRENCY = "RUB"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": [4, 2]}]
LIMIT = 1000

# User preferences for AI
USER_PREFERENCES = "Обязательно две комнаты и две кровати. Хорошие отзывы. Чистота"

# Filters
MIN_PRICE: float | None = 3000.0  # None = no minimum
MAX_PRICE: float | None = 20000.0  # None = no maximum

In [2]:
from etg import ETGClient, GuestRoom, Hotel, HotelContent
from utils import ostrovok_url

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)

In [3]:
async def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = await client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for r in regions[:5]:
        print(f"    - {r['name']} (type: {r['type']}, id: {r['id']})")

    return None

In [4]:
# Find region by city name.
# ETG API requires region_id for hotel search, so we first
# lookup the region ID via suggest_region by city name.
region_id = await find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

Looking up region ID for 'Москва'...
  Found: Москва (RU), region_id=2395

Searching hotels in Москва...
  Dates: 2026-02-02 to 2026-02-04
  Currency: RUB, Limit: 1000


In [5]:
import pandas as pd

from services import search_hotels

# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
search_result = await search_hotels(
    client=client,
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
)

hotels = search_result["hotels"]
total_available = search_result["total_available"]
total_after_filter = search_result["total_after_filter"]

In [6]:
if not hotels:
    print("No hotels found for the given criteria.")
    df_hotels = pd.DataFrame()
else:
    print(f"Found {total_after_filter} hotels after price filter (total available: {total_available})\n")

    # Create DataFrame
    hotels_data = []
    for hotel in hotels:
        rates = hotel.get("rates", [])
        if rates:
            first_rate = rates[0]
            payment_types = first_rate.get("payment_options", {}).get("payment_types", [])
            price = float(payment_types[0].get("show_amount", 0)) if payment_types else None
            room_name = first_rate.get("room_name", "")
            meal = first_rate.get("meal", "")
        else:
            price = None
            room_name = ""
            meal = ""

        hotels_data.append({
            "hotel_id": hotel["id"],
            "hid": hotel["hid"],
            "price": price,
            "currency": CURRENCY,
            "room": room_name[:40] if room_name else "",
            "meal": meal,
        })

    df_hotels = pd.DataFrame(hotels_data)
    df_hotels = df_hotels.sort_values("price", ascending=True).reset_index(drop=True)
    df_hotels.index += 1  # Start from 1

    # Display settings
    pd.set_option("display.max_colwidth", 50)
    pd.set_option("display.max_rows", 100)

df_hotels

Found 302 hotels after price filter (total available: 332)



,hotel_id,hid,price,currency,room,meal
1,na_baumanskoy_hotel,10613383,6074.0,RUB,Четырёхместный номер семейный Эконом-кла,nomeal
2,metro_apartments_5,9990817,6889.0,RUB,Четырёхместный номер Комфорт Двухуровнев,nomeal
3,travel_inn_krasnyie_vorota,8877227,6951.0,RUB,Семейный номер Economy (общая ванная ком,nomeal
4,apartamentyi_sadovoe_koltso_izumrudnaya,8510363,6970.0,RUB,Апартаменты (питание для детей не включе,nomeal
5,zvezda_hotel_6,10437016,7000.0,RUB,Трёхместный номер Comfort (питание для д,nomeal
...,...,...,...,...,...,...
298,katerina_city_hotel,7597146,38000.0,RUB,Двухместный номер Стандарт (двуспальная,breakfast
299,russkie_sezonyi,8848163,38106.0,RUB,Двухместный люкс with Sofa Superior Манс,nomeal
300,hampton_by_hilton_moscow_rogozhsky_val,9770069,38522.0,RUB,Номер с диваном Семейный с кроватью King,breakfast
301,grand_revival_hotel,10004948,38836.0,RUB,Четырёхместный номер Большой Улучшенный,nomeal


In [7]:
from services import batch_get_content

hotel_hids = [h["hid"] for h in hotels]

print(f"[batch_get_content_start] Загрузка контента для {len(hotel_hids)} отелей...")
content_result = await batch_get_content(client, hotel_hids, LANGUAGE)
content_map = content_result["content"]
print(
    f"[batch_get_content_done] Загружен контент для {content_result['total_loaded']} "
    f"из {content_result['total_requested']} отелей ({content_result['total_batches']} батчей)"
)

[batch_get_content_start] Загрузка контента для 302 отелей...
[batch_get_content_done] Загружен контент для 302 из 302 отелей (4 батчей)


In [8]:
content_data = []
for hid, content in content_map.items():
    content_data.append({
        "hid": hid,
        "name": content.get("name", "")[:40],
        "stars": content.get("star_rating", 0),
        "kind": content.get("kind", ""),
        "address": content.get("address", "")[:50],
        "latitude": content.get("latitude"),
        "longitude": content.get("longitude"),
        "check_in": content.get("check_in_time", ""),
        "check_out": content.get("check_out_time", ""),
    })

df_content = pd.DataFrame(content_data)

# Merge with prices from df_hotels
df_full = df_hotels.merge(df_content, on="hid", how="left")
df_full = df_full[["hotel_id", "name", "stars", "kind", "price", "meal", "address"]]
df_full = df_full.sort_values("price", ascending=True).reset_index(drop=True)
df_full.index += 1

df_full

,hotel_id,name,stars,kind,price,meal,address
1,na_baumanskoy_hotel,Отель на Бауманской,0,Hotel,6074.0,nomeal,"улица Фридриха Энгельса, д.31/35, Москва"
2,metro_apartments_5,Апарт-отель Metro rooms,3,Apart-hotel,6889.0,nomeal,"проезд Гостиничный, д.6, корп.2, Москва"
3,travel_inn_krasnyie_vorota,Мини-отель на Комсомольской,0,Mini-hotel,6951.0,nomeal,"улица Новая Басманная, д.10/1, Москва"
4,apartamentyi_sadovoe_koltso_izumrudnaya,Апартаменты Садовое Кольцо Изумрудная,0,Apartment,6970.0,nomeal,"Изумрудная улица, д.6, Москва"
5,zvezda_hotel_6,Городской отель Звезда,0,Hotel,7000.0,nomeal,"г. Люберцы ул. 3-е Почтовое Отделение, д. 44А,..."
...,...,...,...,...,...,...,...
298,katerina_city_hotel,Отель Россо Рива,4,Hotel,38000.0,breakfast,"Шлюзовая набережная, д. 6, стр. 2, Москва"
299,russkie_sezonyi,Бутик-Отель Регул,5,Boutique_and_Design,38106.0,nomeal,"Дегтярный переулок, д. 8/2, Москва"
300,hampton_by_hilton_moscow_rogozhsky_val,Отель Hampton by Hilton Moscow Рогожский,4,Hotel,38522.0,breakfast,"улица Рогожский Вал, дом 10, Москва"
301,grand_revival_hotel,Отель Revival Hotel,3,Hotel,38836.0,nomeal,"улица Петровка 19 стр.3, Москва"


In [9]:
from services import batch_get_reviews, filter_reviews, HotelReviews

print(f"[batch_get_reviews_start] Загрузка отзывов для {len(hotel_hids)} отелей...")
raw_reviews = await batch_get_reviews(client, hotel_hids, LANGUAGE)
reviews_map = filter_reviews(raw_reviews)

total_raw = sum(rd["total_reviews"] for rd in raw_reviews.values())
total_filtered = sum(len(rd["reviews"]) for rd in reviews_map.values())
total_filtered_by_age = sum(rd["filtered_by_age"] for rd in reviews_map.values())
hotels_with_reviews = len(reviews_map)

# Calculate average rating across all hotels
all_avg_ratings = [rd["avg_rating"] for rd in reviews_map.values() if rd["avg_rating"] is not None]
overall_avg = sum(all_avg_ratings) / len(all_avg_ratings) if all_avg_ratings else 0

print(
    f"[batch_get_reviews_done] Всего {hotels_with_reviews} отелей с отзывами из {len(hotel_hids)}"
)
print(
    f"  Обработано {total_raw} отзывов → {total_filtered} релевантных "
    f"(отсечено по давности: {total_filtered_by_age})"
)
print(f"  Средний рейтинг: {overall_avg:.1f}/10")

[batch_get_reviews_start] Загрузка отзывов для 302 отелей...
[batch_get_reviews_done] Всего 299 отелей с отзывами из 302
  Обработано 46206 отзывов → 11880 релевантных (отсечено по давности: 6787)
  Средний рейтинг: 8.5/10


In [10]:
# Create DataFrame with reviews summary
reviews_data = []
for hid, data in reviews_map.items():
    hotel_id = next((h["id"] for h in hotels if h["hid"] == hid), "")
    detailed = data["detailed_averages"]
    reviews_data.append({
        "hotel_id": hotel_id,
        "hid": hid,
        "total": data["total_reviews"],
        "avg_rating": data["avg_rating"],
        "cleanness": detailed["cleanness"],
        "location": detailed["location"],
        "price": detailed["price"],
        "services": detailed["services"],
        "room": detailed["room"],
        "meal": detailed["meal"],
        "wifi": detailed["wifi"],
        "hygiene": detailed["hygiene"],
    })

df_reviews = pd.DataFrame(reviews_data)
df_reviews = df_reviews.sort_values("avg_rating", ascending=False, na_position="last").reset_index(drop=True)
df_reviews.index += 1

# Merge with hotel info
df_reviews_full = df_reviews.merge(
    df_content[["hid", "name", "stars"]],
    on="hid",
    how="left"
)
df_reviews_full = df_reviews_full[[
    "hotel_id", "name", "stars", "total", "avg_rating",
    "cleanness", "location", "room", "services", "price", "meal", "wifi", "hygiene"
]]


def show_reviews(hotel_id: str, limit: int = 5) -> None:
    """Show reviews for a hotel."""
    hid = next((h["hid"] for h in hotels if h["id"] == hotel_id), None)
    if not hid:
        print(f"Hotel '{hotel_id}' not found")
        return
    
    data = reviews_map.get(hid)
    if not data:
        print(f"No reviews for hotel '{hotel_id}'")
        return
    
    hotel_name = content_map.get(hid, {}).get("name", hotel_id)
    avg = data["avg_rating"]
    d = data["detailed_averages"]
    
    print(f"{'='*70}")
    print(f"{hotel_name}")
    print(f"Total: {data['total_reviews']} reviews | Avg rating: {avg}/10" if avg else f"Total: {data['total_reviews']} reviews")
    print(f"{'='*70}")
    print(f"Detailed scores:")
    print(f"  cleanness: {d['cleanness']}  location: {d['location']}  room: {d['room']}  services: {d['services']}")
    print(f"  price: {d['price']}  meal: {d['meal']}  wifi: {d['wifi']}  hygiene: {d['hygiene']}")
    print(f"{'='*70}\n")
    
    reviews = data["reviews"][:limit]
    for r in reviews:
        rating = r["rating"]
        date = r["created"][:10]
        lang = r.get("_lang", "?")
        plus = r.get("review_plus", "").strip()
        minus = r.get("review_minus", "").strip()
        print(f"[{rating}/10] {date} [{lang}]")
        if plus:
            print(f"  + {plus[:300]}")
        if minus:
            print(f"  - {minus[:300]}")
        print()


df_reviews_full

,hotel_id,name,stars,total,avg_rating,cleanness,location,room,services,price,meal,wifi,hygiene
0,bobolink_cold_na_sokolinoy_gore_lodging_house,Апартаменты двухуровневые Cosmo Star на,0,10,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
1,scandi_na_tverskoy_15_apartments,Апартаменты Scandi на Тверской 15,0,7,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
2,great_apart_na_arbate_1,Great apart Duplex на Арбате,0,1,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
3,sovremennaya_i_stilnaya_kvartira_ryadom_s_kremlem,Квартира GM Apartments рядом с Кремлем н,0,3,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
4,osennyaya_apartments,Апартаменты Осенняя,0,4,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,zvezda_hotel_6,Городской отель Звезда,0,96,4.8,4.6,5.1,4.0,5.0,4.9,6.2,10.0,5.0
295,hotel_apelsin_vnukovo,Апельсин Гостевой Дом во Внуково,0,35,4.4,4.1,5.9,3.6,4.0,3.8,4.8,10.0,NaN
296,moskva_siti_apex_42_etazh_2_komnaty_apartments,Апартаменты Москва Сити Apex 42 этаж 2 к,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,hotelroom24_u_metro_belorusskaya_apartments_3,Апартаменты HotelRoom24 у метро белорусс,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Example: view reviews for a specific hotel
# show_reviews("four_seasons_st_petersburg")       # 5 reviews
# show_reviews("four_seasons_st_petersburg", 10)   # 10 reviews

show_reviews("four_seasons_st_petersburg", limit=3)

Hotel 'four_seasons_st_petersburg' not found


In [12]:
from services import combine_hotels_data, HotelFull

combined = combine_hotels_data(hotels, content_map, reviews_map)
print(f"Combined {len(combined)} hotels with content and reviews")

Combined 302 hotels with content and reviews


In [13]:
import json

from services import estimate_tokens, prepare_hotel_for_llm, presort_hotels, score_hotels

# Estimate tokens before presort
hotels_for_llm_all = [prepare_hotel_for_llm(h) for h in combined]
tokens_before = estimate_tokens(json.dumps(hotels_for_llm_all, ensure_ascii=False))

# Pre-sort by hotel kind tier and prescore, limit to top 100 for LLM scoring
PRESORT_LIMIT = 300
top_hotels = presort_hotels(combined, reviews_map, limit=PRESORT_LIMIT)

# Estimate tokens after presort
hotels_for_llm_top = [prepare_hotel_for_llm(h) for h in top_hotels]
tokens_after = estimate_tokens(json.dumps(hotels_for_llm_top, ensure_ascii=False))

print(f"[presort_done] {len(combined)} отелей → {len(top_hotels)} (лимит {PRESORT_LIMIT})")
print(f"  Токены: ~{tokens_before:,} → ~{tokens_after:,} (экономия {tokens_before - tokens_after:,})")

[presort_done] 302 отелей → 300 (лимит 300)
  Токены: ~365,332 → ~362,934 (экономия 2,398)


In [14]:
import time

from services import finalize_scored_hotels, score_hotels

# Score hotels using single LLM request
# Returns top 10 scored hotels with summary explaining the selection
print(f"[scoring_start] Scoring {len(top_hotels)} hotels...")
start_time = time.time()

scoring_result = await score_hotels(
    top_hotels,
    USER_PREFERENCES,
    guests=GUESTS,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    currency=CURRENCY,
    top_count=10
)

elapsed = time.time() - start_time

if scoring_result["error"]:
    print(f"\n❌ ERROR: {scoring_result['error']}")
    scored_hotels = None
else:
    scoring_results = scoring_result["results"]
    print(f"[scoring_done] {len(scoring_results)} hotels scored — {elapsed:.1f}s")
    print(f"  Estimated tokens: ~{scoring_result['estimated_tokens']:,}")
    print(f"\n📋 Summary:\n{scoring_result['summary']}")
    
    # Finalize scored hotels - merge scoring results with full hotel data
    scored_hotels = finalize_scored_hotels(combined, scoring_results)
    print(f"\n[finalize_done] {len(scored_hotels)} hotels with complete data")

[scoring_start] Scoring 300 hotels...
[scoring_done] 10 hotels scored — 38.0s
  Estimated tokens: ~365,711

📋 Summary:
Рынок краткосрочной аренды и отелей Москвы в анализируемом диапазоне от 3000 до 20000 рублей за ночь представлен преимущественно современными апарт-отелями и бутик-отелями, сосредоточенными как в историческом центре, так и в деловых кластерах. Средняя стоимость качественного семейного размещения с двумя комнатами или кухонной зоной составляет около 12000–15000 рублей. В процессе отбора приоритет отдавался объектам с рейтингом выше 8.5, обеспечивающим безупречную чистоту и наличие полноценных спальных мест для детей. Например, отель «Восход» (ID: hotel_voshod, рейтинг 6.8) был исключен из рекомендаций из-за низких оценок за гигиену и состояние номерного фонда. Также не вошли в топ объекты с устаревшим ремонтом, такие как «Аэрополис» (ID: gostinitsa_aeropolis, рейтинг 7.9), так как за аналогичную стоимость рынок предлагает более современные и комфортные апартаменты. Итог

In [15]:
from services import HotelScored


def display_top_hotels(
    scored_hotels: list[HotelScored],
    top_n: int = 10,
) -> pd.DataFrame:
    """Display top N scored hotels with details and Ostrovok links."""
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    data = []
    for i, hotel in enumerate(scored_hotels[:top_n], 1):
        hotel_id = hotel["id"]
        name = hotel["name"]
        hid = hotel["hid"]
        kind = hotel.get("kind", "")
        score = hotel["score"]
        reasons = hotel.get("top_reasons", [])
        penalties = hotel.get("score_penalties", [])
        selected_hash = hotel.get("selected_rate_hash", "")
        
        # Find selected rate by hash
        rates = hotel.get("rates", [])
        selected_rate = next((r for r in rates if r.get("match_hash") == selected_hash), None)
        
        # Get rate details
        if selected_rate:
            room_name = selected_rate.get("room_name", "")[:50]
            meal_data = selected_rate.get("meal_data", {})
            meal = meal_data.get("value", selected_rate.get("meal", ""))
            
            # Calculate prices from daily_prices
            daily_prices = selected_rate.get("daily_prices", [])
            if daily_prices:
                # Convert string prices to float and sum
                total_price = sum(float(p) for p in daily_prices)
                num_nights = len(daily_prices)
                avg_price_per_night = total_price / num_nights if num_nights > 0 else 0
                
                # Get currency
                pt = selected_rate.get("payment_options", {}).get("payment_types", [])
                currency = pt[0].get("show_currency_code", "") if pt else ""
                
                total_price_str = f"{total_price:.0f} {currency}"
                avg_price_str = f"{avg_price_per_night:.0f} {currency}"
            else:
                # Fallback to payment_types if daily_prices not available
                pt = selected_rate.get("payment_options", {}).get("payment_types", [])
                if pt:
                    total_price = float(pt[0].get("show_amount", 0))
                    currency = pt[0].get("show_currency_code", "")
                    total_price_str = f"{total_price:.0f} {currency}"
                    avg_price_str = f"{total_price:.0f} {currency}"
                else:
                    total_price_str = "N/A"
                    avg_price_str = "N/A"
        else:
            room_name = "N/A"
            meal = "N/A"
            total_price_str = "N/A"
            avg_price_str = "N/A"
        
        # Get reviews data
        reviews = hotel.get("reviews")
        avg_rating = reviews.get("avg_rating") if reviews else None
        detailed = reviews.get("detailed_averages", {}) if reviews else {}
        
        # Generate Ostrovok URL
        url = ostrovok_url(
            hotel_id=hotel_id,
            hid=hid,
            checkin=CHECKIN_DATE,
            checkout=CHECKOUT_DATE,
            guests=GUESTS,
            region_id=region_id,
        )

        # Print detailed info
        print(f"{i}. {name} [{kind}]")
        print(f"   Score: {score}/100 | Rating: {avg_rating}/10" if avg_rating else f"   Score: {score}/100")
        print(f"   Room: {room_name}")
        print(f"   Total: {total_price_str} | Avg per night: {avg_price_str} | Meal: {meal}")
        if reasons:
            print(f"   + {'; '.join(reasons[:3])}")
        if penalties:
            print(f"   - {'; '.join(penalties[:5])}")
        print(f"   🔗 {url}")
        print()
        
        # Collect for DataFrame
        data.append({
            "name": name[:35],
            "kind": kind,
            "room": room_name[:30],
            "total": total_price_str,
            "avg/night": avg_price_str,
            "meal": meal,
            "score": score,
            "rating": avg_rating,
            "clean": detailed.get("cleanness"),
            "url": url,
        })
    
    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    selected = min(top_n, len(scored_hotels))
    print(f"Всего найдено {len(combined)} отелей на эти даты.")
    print(f"Подобраны лучшие {selected} по вашим критериям.")
    return df


pd.set_option("display.max_colwidth", 100)
display_top_hotels(scored_hotels, top_n=10)


TOP 10 HOTELS

1. Апартаменты Family Apartment с панорамным видом на город [Apartment]
   Score: 98/100 | Rating: 9.9/10
   Room: Трёхместный номер Superior с красивым видом из окн
   Total: 18050 RUB | Avg per night: 9025 RUB | Meal: nomeal
   + Исключительный рейтинг (9.9/10) подтверждает высочайшее качество сервиса и чистоты; Панорамный вид на город и близость к парку развлечений «Остров Мечты»; Полностью укомплектованная кухня и стиральная машина для комфортного семейного отдыха
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid10155362/family_apartment_tekhnopark_apartments/?dates=02.02.2026-04.02.2026&guests=2and4.2&q=2395

2. Городской отель Дом Купца Бавыкина [Hotel]
   Score: 96/100 | Rating: 9.6/10
   Room: Четырёхместный номер с проекционным кинотеатром се
   Total: 17687 RUB | Avg per night: 8844 RUB | Meal: nomeal
   + Уникальное расположение в историческом здании 1840-х годов (объект культурного наследия); Высокий рейтинг чистоты (9.8/10) и гостеприимства; Наличие проекцио

,name,kind,room,total,avg/night,meal,score,rating,clean,url
1,Апартаменты Family Apartment с пано,Apartment,Трёхместный номер Superior с к,18050 RUB,9025 RUB,nomeal,98,9.9,9.8,https://ostrovok.ru/hotel/russia/moscow/mid10155362/family_apartment_tekhnopark_apartments/?date...
2,Городской отель Дом Купца Бавыкина,Hotel,Четырёхместный номер с проекци,17687 RUB,8844 RUB,nomeal,96,9.6,9.8,https://ostrovok.ru/hotel/russia/moscow/mid11347687/dom_kuptsa_bavykina_mini_hotel/?dates=02.02....
3,Гостиница Raido.Moscow Sky,Hotel,Двухместный номер Deluxe с вид,20653 RUB,10326 RUB,nomeal,95,9.6,9.7,https://ostrovok.ru/hotel/russia/moscow/mid11295321/raidomoscow_sky_apart_hotel/?dates=02.02.202...
4,Отель Покровка 6,Hotel,Двухместный номер Big Мансарда,20520 RUB,10260 RUB,nomeal,94,9.5,9.5,https://ostrovok.ru/hotel/russia/moscow/mid8742888/pokrovka_6_hotel/?dates=02.02.2026-04.02.2026...
5,Апарт-отель Axis.Moscow Heritage,Apart-hotel,Двухместный номер Deluxe Манса,18496 RUB,9248 RUB,nomeal,93,9.4,9.6,https://ostrovok.ru/hotel/russia/moscow/mid11292139/axismoscow_heritage_aparthotel/?dates=02.02....
6,Апарт-отель Idera,Hotel,Двухместный номер Улучшенный Д,14432 RUB,7216 RUB,nomeal,92,9.3,9.6,https://ostrovok.ru/hotel/russia/moscow/mid13106596/idera_hotel/?dates=02.02.2026-04.02.2026&gue...
7,Отель Авион Внуково,Hotel,Четырёхместный номер Deluxe с,18300 RUB,9150 RUB,nomeal,91,9.2,9.4,https://ostrovok.ru/hotel/russia/moscow/mid6426482/vnukovo_otel_green_palace/?dates=02.02.2026-0...
8,Отель Облака,Hotel,Четырёхместный номер Superior,10600 RUB,5300 RUB,nomeal,90,9.3,9.2,https://ostrovok.ru/hotel/russia/moscow/mid6692379/tsvetnoe_oblako/?dates=02.02.2026-04.02.2026&...
9,Гостиница Апарт-отель Raido.Moscow,Hotel,Studio Suite (питание для дете,17808 RUB,8904 RUB,nomeal,89,9.3,9.3,https://ostrovok.ru/hotel/russia/moscow/mid10292809/raldomoscow_apart_hotel/?dates=02.02.2026-04...
10,Отель WineWood Moscow,Hotel,Двухместный номер Делюкс (двус,18000 RUB,9000 RUB,nomeal,88,8.9,8.9,https://ostrovok.ru/hotel/russia/moscow/mid8854262/winewood_moscow_lounge_hotel/?dates=02.02.202...
